<a href="https://colab.research.google.com/github/vifirsanova/empi/blob/main/demos/greetings_scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Приветственный сценарий работы чат-бота Эмпик**

*Эмпик - модель Conversational AI для инклюзивного образования. Первый запуск модели сопровождается приветственным сценарием, который вы можете опробовать в этом ноутбуке.*

**Эмпик анализирует, распознает и сохраняет:**
- имя пользователя
  - распознается автоматически с помощью NER
- настройки мобильного приложения EMPI AI
  - распознаются автоматически с помощью рекурсивного алгоритма поиска по графу - [базе знаний EMPI](https://github.com/vifirsanova/empi/blob/main/demos/graph_crowdsoursing_ui.ipynb)
- историю общения с пользователем
  - модель ведет лог для интерпретации результатов работы чат-бота в исследовательских целях (Explainable AI)

**Отказ от ответственности**

Автор демо-версии ПО не несет ответственности за точность, полноту или качество предоставленной в выдачах ПО информации. Никакие претензии за материальный или нематериальный ущерб, вызванный использованием или неиспользованием предоставленной информации не принимаются.

**Обратная связь**

Автор демо-версии: [Виктория Фирсанова](https://vifirsanova.github.io/).
Свою обратную связь вы можете отправить в Telegram @vifirsanova или на почту vifirsanova@gmail.com. Принимаются любые пожелания и предложения по разработке продукта.

In [ ]:
#@title **Приветственный сценарий работы чат-бота Эмпик (демо-версия)**

# загрузка библиотек
!pip install spacy
!spacy download ru_core_news_sm
!pip install gensim

# загрузка модулей
import re
from gensim.models import Word2Vec
import spacy
import json
from IPython.display import clear_output

# граф знаний: открыть
with open('/content/drive/MyDrive/updated.json', 'r') as f:
  data = json.load(f)

class NLProcessor:
  """
  Модуль для предобработки текстов на естественном языке
  """
  def __init__(self, model, nlp):
    self.model = model # модель для эмбеддингов
    self.nlp = nlp # модель для обработки текстов
    self.named_entities = [] # для функции ner_extract

  def tokenize(self, text):
    return self.nlp(text)

  def get_tokens(self, text, lower=True):
    _text = self.tokenize(re.sub(r'[^\w\s-]', '', text))
    return [token.text.lower() for token in _text] if lower == True else [token.text for token in _text]

  def _synonyms(self, word, topn):
    # n самых похожих по вектору слов
    try:
      synonyms = [elem[0] for elem in self.model.wv.most_similar(word)[:topn]]
      return synonyms
    except KeyError:
      return word

  def find_synonyms(self, text, topn=5):
    syns = []
    tokens = self.get_tokens(text)
    for token in tokens:
      for s in self._synonyms(token, topn=topn):
        syns.append(s)
    return syns

  def ner_extract(self, text):
    tokenized_text = self.tokenize(text)
    return [(ent.text, ent.label_) for ent in tokenized_text.ents]

class Search:
  """
  Модуль для поиска по графу
  """
  def __init__(self, nlprocessor):
    self.path = []
    self.matched = set()
    self.nlprocessor = nlprocessor # класс NLProcessor

  def graph_search(self, data, search):
    for k in data:
      if search in k.split():
        self.path.append(data[k])
        self.matched.add(search)
      if isinstance(data[k], dict):
        self.graph_search(data=data[k], search=search)
        self.matched.add(search)
      else:
        if isinstance(data[k], str):
          if search in data[k].split():
            self.path.append(data[k])
            self.matched.add(search)
        else:
          for elem in data[k]:
            if search in elem.split():
              self.path.append(data[k])
              self.matched.add(search)

  def searching(self, data, query, exact=False):
    tokenized_query = self.nlprocessor.get_tokens(query)
    synonyms = self.nlprocessor.find_synonyms(query)

    for word in tokenized_query:
      self.graph_search(data, word)

    if exact == False:
      for word in synonyms:
        self.graph_search(data, word)
    # возвращает путь к найденному объекту и все метчи из входных данных
    return {'path': self.path, 'matches': list(self.matched)}

class Greetings():
  """
  Initial Generation: этот сценарий запускается при первом общении с пользователем
  Сценарий 1: приветствие и распознавание специальных потребностей пользователя
  Модель извлекает информацию из ответа пользователя, ищет совпадения в графовой базе знаний в ветке "дизайн",
  передает команду в пользовательский интерфейс, чтобы включить специальные возможности
  - настройка мобильного приложения EMPI AI
  - создание первичного блока с пользовательской информацией
  - имя пользователя
  - специальные настройки
  - история общения с диалоговой системой
  """
  def __init__(self, nlprocessor, search):
    self.nlprocessor = nlprocessor
    self.search = search
    self.log = '' # история общения с пользователем

  def update_log(self):
    self.log += self.response
    self.log += self.prompt

  def init_gen(self):
    # извлекаем именованные сущности из ответа пользователя
    self.response = "ЭМПИК:\nПривет! Как тебя зовут?\n"
    self.prompt = input(self.response)
    username_ner = nlprocessor.ner_extract(self.prompt)
    # извлекаем первое имя собственное из найденных
    global username
    username = [x[0] for x in username_ner if x[1] == 'PER'][0]
    self.update_log()

    # запрашиваем информацию о нуждах пользователя
    self.response = f"\nЭМПИК:\nРасскажи мне о себе: что тебе нужно для комфортного общения со мной?\
    \nНапример, озвучивание текста, крупный шрифт или упрощенный язык.\n{username}: "
    self.prompt = input(self.response)
    self.update_log()

    # жадный поиск по графу (паттерн-матчинг)
    global setting
    setting = search.searching(data=data['empi']['дизайн'], query=self.prompt, exact=True)["path"]
    if len(setting) > 0:
      self.response = f"\nЭМПИК:\nОтлично! Я тебя понял. Включаю режим: <{setting[0]}>"
      print(self.response)
    else:
      self.response = '\nЭМПИК:\nОтлично! Давай общаться.'
      print(self.response)
    self.update_log()

  def user_block(self):
    block = dict()
    block['username'] = username
    block['init_log'] = greetings.log
    block['setting'] = setting

    with open(f"{username}_block.json", "w", encoding="utf-8") as f:
        json.dump(block, f, ensure_ascii=False)

# инициализация компонентов для первичной генерации:
# процессор, поисковая система, привественный сценарий
model = Word2Vec.load('/content/drive/MyDrive/word2vec_model(1).bin')
nlp = spacy.load("ru_core_news_sm")
nlprocessor = NLProcessor(model, nlp)
search = Search(nlprocessor)
greetings = Greetings(nlprocessor, search)

# тестирование
clear_output()
greetings.init_gen()
greetings.user_block()

ЭМПИК:
Привет! Как тебя зовут?
Приветик! Меня зовут Аня.

ЭМПИК:
Расскажи мне о себе: что тебе нужно для комфортного общения со мной?    
Например, озвучивание текста, крупный шрифт или упрощенный язык.
Аня: Думаю, что мне понадобится распознавание речи.

ЭМПИК:
Отлично! Я тебя понял. Включаю режим: <распознавание речи>


In [ ]:
#@title **Результат обработки приветственного сценария чат-бота:** блок с персональной пользовательской информацией

from pprint import pprint

with open(f"{username}_block.json", 'r') as f:
  block = json.load(f)

pprint(block)

{'init_log': 'ЭМПИК:\n'
             'Привет! Как тебя зовут?\n'
             'Приветик! Меня зовут Аня.\n'
             'ЭМПИК:\n'
             'Расскажи мне о себе: что тебе нужно для комфортного общения со '
             'мной?    \n'
             'Например, озвучивание текста, крупный шрифт или упрощенный '
             'язык.\n'
             'Аня: Думаю, что мне понадобится распознавание речи.\n'
             'ЭМПИК:\n'
             'Отлично! Я тебя понял. Включаю режим: <распознавание речи>Думаю, '
             'что мне понадобится распознавание речи.',
 'setting': ['распознавание речи',
             'распознавание речи',
             {'для кого': ['рас',
                           'афазия',
                           'нарушения процесса порождения речи'],
              'принцип работы': 'технология автоматического воспроизведения '
                                'текста, например, функция “прямая речь” в '
                                'iphone',
              'технология': 't

Персональный блок содержит имя пользователя `username`, настройки приложения EMPI AI `setting`, историю общения с пользователем `init log`. Настройки позволяют обеспечить **доступность EMPI AI**, например, включить режим надиктовывания и озвучивания текста на экране для людей с нарушениями зрения.